In [1]:
using Pkg; Pkg.activate("..")
using Geom4hep

  Activating environment at `~/Development/VecGeom/Geom4hep/Project.toml`
┌ Info: Precompiling Geom4hep [eb5d0804-93e0-431a-a4d4-b4f95b95575a]
└ @ Base loading.jl:1342


In [2]:
using Test
pzero = Point3(0., 0., 0.)
ponxside, ponyside, ponzside = Point3(20., 0., 0.), Point3(0., 30., 0.), Point3(0., 0., 40.)
ponmxside, ponmyside, ponmzside = Point3(-20., 0., 0.), Point3(0., -30., 0.), Point3(0., 0., -40.)
ponzsidey, ponmzsidey = Point3(0., 25., 40.), Point3(0., 25., -40.)

b1 = Box(20., 30., 40.)
b2 = Box(10., 10., 10.)
box3 = Box(0.14999999999999999, 24.707000000000001, 22.699999999999999)

pbigx = Point3(100., .0, 0.)
pbigy = Point3(0., 100., 0.)
pbigz = Point3(0., 0., 100.)
pbigmx = Point3(-100., 0., 0.)
pbigmy = Point3(0., -100., 0.)
pbigmz = Point3(0., 0., -100.)

vx = Vector3(1., 0., 0.)
vy = Vector3(0., 1., 0.)
vz = Vector3(0., 0., 1.)
vmx = Vector3(-1., 0., 0.)
vmy = Vector3(0., -1., 0.)
vmz = Vector3(0., 0., -1.)
vxy = Vector3(1/√2, 1/√2, 0.)
vmxy = Vector3(-1/√2, 1/√2, 0.)
vmxmy = Vector3(-1/√2, -1/√2, 0.)
vxmy = Vector3(1/√2, -1/√2, 0.)
vxmz = Vector3(1/√2, 0., -1/√2)


3-element StaticArrays.SVector{3, Float64} with indices SOneTo(3):
  0.7071067811865475
  0.0
 -0.7071067811865475

In [3]:
using LinearAlgebra
bx = Box(1.,200., 200.)
testp = Point3(1.,0.,0.) + 6e-15 * vx
testv = normalize(Vector3(0.,1.,1.) - 4e-6 * vx)
dist = distanceToIn(bx, testp, testv)
@test isapprox(dist, 0, atol=1e-6)
dist = distanceToOut(bx, testp, testv)
@show mynormal(bx, testp + dist*testv)

normal(bx, testp + dist * testv) = nothing


3-element Point3{Float64} with indices SOneTo(3):
  1.0000028284271307
 -0.7071067811837191
 -0.7071067811837191

In [7]:
sqrt(eps())*norm(dist)

3.15848711018499e-17

In [134]:
using LinearAlgebra
# DistanceToIn(P,V)
@test distanceToIn(b1, pbigx, vmx) ≈ 80
@test distanceToIn(b1, pbigmx, vx) ≈ 80
@test distanceToIn(b1, pbigy, vmy) ≈ 70
@test distanceToIn(b1, pbigmy, vy) ≈ 70
@test distanceToIn(b1, pbigz, vmz) ≈ 60
@test distanceToIn(b1, pbigmz, vz) ≈ 60

# testing a few special directions ( with special zero values )
# important since we sometimes operate with sign bits etc.
@test distanceToIn(b1, Point3(0., 0., -50.), Vector3(-0., 0., 1.)) ≈ 10
@test distanceToIn(b1, Point3(0., 0., -50.), Vector3( 0.,-0., 1.)) ≈ 10
@test distanceToIn(b1, Point3(0., 0.,  50.), Vector3( 0., 0.,-1.)) ≈ 10
@test distanceToIn(b1, Point3(0., -40., 0.), Vector3( 0., 1., 0.)) ≈ 10
@test distanceToIn(b1, Point3(0., -40., 0.), Vector3( 0., 1.,-0.)) ≈ 10
@test distanceToIn(b1, Point3(0., -40., 0.), Vector3(-0., 1., 0.)) ≈ 10
@test distanceToIn(b1, Point3(-30., 0., 0.), Vector3( 1.,-0., 0.)) ≈ 10
@test distanceToIn(b1, Point3( 30,  0., 0.), Vector3(-1., 0.,-0.)) ≈ 10
@test distanceToIn(b1, Point3(-30., 0., 0.), Vector3( 1.,-0.,-0.)) ≈ 10

@test distanceToIn(b1, pbigx, vxy) ≈ Inf
@test distanceToIn(b1, pbigmx, vxy) ≈ Inf

pJohnXZ = Point3(9., 0., 12.)
@test distanceToIn(b2, pJohnXZ, vxmz) ≈ Inf
pJohnXY = Point3(12., 9., 0.)
@test distanceToIn(b2, pJohnXY, vxmy) ≈ Inf
@test distanceToIn(b2, pJohnXY, vmx) ≈ 2
pMyXY = Point3(32., -11., 0.)
@test distanceToIn(b2, pMyXY, vmxy) ≈ Inf
@test distanceToIn(b1, Point3(-25., -35., 0.), vx) ≈ Inf
@test distanceToIn(b1, Point3(-25., -35., 0.), vy) ≈ Inf
@test distanceToIn(b2, pJohnXY, vmx) ≈ 2

tempDir = normalize(Vector3(-0.76165597579890043, 0.64364445891356026, -0.074515708658524193))
d = distanceToIn(box3, Point3(0.15000000185, -22.048743592955137, 2.4268539333219472), tempDir)
@test isapprox(d, 0, atol=1e-6)

# testing tolerance of DistanceToIn
b4 = Box(5., 5., 5.)
# a point very slightly inside should return 0
tempDir = normalize(Vector3(0.76315134679548990437, 0.53698876104646497964, -0.35950395323836459305))
@test distanceToIn(b4, Point3(-3.0087437277453119577, -5.0+kTolerance/2, 4.8935648380409944025), tempDir) <= 0.0


# a point on the surface pointing outside must return infinity length
@test distanceToIn(b2,Point3(10., 0., 0.), Vector3(1., 0., 0.)) == Inf
@test distanceToIn(b2,Point3(10. - 0.9*eps(), 0., 0.), Vector3(1.,0.,0.)) >= Inf


distance = 80.0
distout = 120.0
distsurf = 80.0
distance = 80.0
distout = 120.0
distsurf = 80.0
distance = 70.0
distout = 130.0
distsurf = 70.0
distance = 70.0
distout = 130.0
distsurf = 70.0
distance = 60.0
distout = 140.0
distsurf = 60.0
distance = 60.0
distout = 140.0
distsurf = 60.0
distance = 10.0
distout = 90.0
distsurf = 10.0
distance = 10.0
distout = 90.0
distsurf = 10.0
distance = 10.0
distout = 90.0
distsurf = 10.0
distance = 10.0
distout = 70.0
distsurf = 10.0
distance = 10.0
distout = 70.0
distsurf = 10.0
distance = 10.0
distout = 70.0
distsurf = 10.0
distance = 10.0
distout = 50.0
distsurf = 10.0
distance = 10.0
distout = 50.0
distsurf = 10.0
distance = 10.0
distout = 50.0
distsurf = 10.0
distance = -42.42640687119285
distout = -113.13708498984761
distsurf = 80.0
distance = 113.13708498984761
distout = 42.42640687119285
distsurf = 80.0
distance = 2.8284271247461903
distout = 1.4142135623730951
distsurf = 2.0
distance = -1.4142135623730951
distout = -2.8284271247461903
dist

Test Passed

In [105]:
kTolerance = 1e-9
kTolerance/2


5.0e-10

In [131]:
@show pJohnXZ, vxmz
distanceToIn(b2, pJohnXZ, vxmz)

(pJohnXZ, vxmz) = ([9.0, 0.0, 12.0], [0.7071067811865475, 0.0, -0.7071067811865475])
distance = 2.8284271247461903
distout = 1.4142135623730951
distsurf = 2.0


Inf

In [124]:
(distance >= distout || distout <= kTolerance/2 || distsurf <= kTolerance/2)

LoadError: UndefVarError: distance not defined